In [3]:
# Import Dependencies
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization

In [4]:
# Set up Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Change Directory
%cd /content/drive/MyDrive/pet_expressions_data

/content/drive/MyDrive/pet_expressions_data


In [6]:
# Define the path to the dataset folders
happy_folder = "/content/drive/MyDrive/pet_expressions_data/happy"
sad_folder = "/content/drive/MyDrive/pet_expressions_data/sad"
angry_folder = "/content/drive/MyDrive/pet_expressions_data/angry"
other_folder = "/content/drive/MyDrive/pet_expressions_data/other"

In [7]:
# Check the number of pictures in each 'emotion' folder, ideally we would want this to be equal
# Define function to count picturues within a folder, takes path as input

def count_pics(folder_path):
    picture_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    picture_count = 0

    # Check if the provided folder path exists
    if not os.path.exists(folder_path):
        print("Folder path does not exist.")
        return 0

    # Get a list of files in the folder
    files = os.listdir(folder_path)

    # Count pictures with valid extensions
    for file in files:
        _, extension = os.path.splitext(file)
        if extension.lower() in picture_extensions:
            picture_count += 1

    return picture_count

In [8]:
# Happy
happy_picture_count = count_pics(happy_folder)
print(f"Number of pictures in the HAPPY folder: {happy_picture_count}")

Number of pictures in the HAPPY folder: 112


In [9]:
# Sad
sad_picture_count = count_pics(sad_folder)
print(f"Number of pictures in the SAD folder: {sad_picture_count}")

Number of pictures in the SAD folder: 105


In [10]:
# Angry
angry_picture_count = count_pics(angry_folder)
print(f"Number of pictures in the ANGRY folder: {angry_picture_count}")

Number of pictures in the ANGRY folder: 94


In [11]:
 # Other
 other_picture_count = count_pics(other_folder)
print(f"Number of pictures in the  OTHER folder: {other_picture_count}")

Number of pictures in the  OTHER folder: 59


In [12]:
# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images

In [13]:
# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)
other_images = load_images_from_folder(other_folder)

In [14]:
# Create labels for each emotion category
happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)
other_labels = [3] * len(other_images) # 3???? not sure if thats correct

In [15]:
# Concatenate images and labels
x = np.array(happy_images + sad_images + angry_images + other_images)
y = np.array(happy_labels + sad_labels + angry_labels + other_labels)

In [16]:
# Normalize pixel values to range [0, 1]
x = x.astype('float32') / 255.0

In [17]:
# One-hot encode the labels
y = tf.keras.utils.to_categorical(y, 4)

In [18]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [72]:
# Build the CNN model
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(512, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))


In [73]:
print(model.summary())


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_75 (Conv2D)          (None, 46, 46, 64)        640       
                                                                 
 batch_normalization_37 (Bat  (None, 46, 46, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_76 (Conv2D)          (None, 44, 44, 64)        36928     
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 22, 22, 64)       0         
 g2D)                                                            
                                                                 
 dropout_44 (Dropout)        (None, 22, 22, 64)        0         
                                                                 
 conv2d_77 (Conv2D)          (None, 20, 20, 128)     

In [74]:
# Compile the model with class weights
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

In [75]:
# Add early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [76]:
# Calculate class weights
total_samples = len(y_train)
class_weights = {0: total_samples / np.sum(y_train[:, 0]),
                 1: total_samples / np.sum(y_train[:, 1]),
                 2: total_samples / np.sum(y_train[:, 2]),
                 3: total_samples / np.sum(y_train[:, 3])}

In [77]:
# Train/Fit the model with class weights
history = model.fit(x_train.reshape(-1, 48, 48, 1), y_train, batch_size=32, epochs=10, validation_split=0.1, class_weight=class_weights,verbose=0)

In [78]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(x_test.reshape(-1, 48, 48, 1), y_test)
losstr, accuracytr = model.evaluate(x_train.reshape(-1, 48, 48, 1), y_train)

10/10 [==============================] - 2s 240ms/step - loss: 1.3779 - accuracy: 0.2601


In [79]:
print(f"Test accuracy: {accuracy*100:.2f}%")
print(f"Train accuracy: {accuracytr*100:.2f}%")

Test accuracy: 25.68%
Train accuracy: 26.01%


In [ ]:
# Save the trained model
model.save("pet_expressions_model.h5")

In [ ]:
##################################################################################

In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model('pet_expressions_model.h5')

In [ ]:
# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images

In [ ]:
# Load a custom test image
custom_test_image_path = ""

custom_test_image = cv2.imread(custom_test_image_path)
custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (48, 48))
custom_test_image = custom_test_image.astype('float32') / 255.0

In [ ]:
# Reshape the image to match the model input shape
custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

In [ ]:
# Make predictions on the custom test image
prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

In [ ]:
emotion_label = np.argmax(prediction[0])

In [ ]:
# Map the predicted label to emotion class
emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry', 3: 'feeling some type of way...'}
predicted_emotion = emotion_classes[emotion_label]

In [ ]:
# Print the custom test image and its predicted label
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry,feeling some type of way...]: {prediction_prob}")

In [ ]:
import matplotlib.pyplot as plt

#Display the custom test image using matplotlib
plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
from PIL import Image
# Display the original custom test image using PIL
img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()